# Start

- This a first step into extracting the chapters font size for a more structured & enriched chunking. The chapter font size is saved in a json file and can be used in the future to extract the chapters more accurately.

In [ ]:
import fitz

if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

# Functions

In [ ]:
# DOC_PATH = r".\documents\curated\American-Red-Cross-First-Aid-CPR-AED-Participants-Manual-CURATED.pdf"
# DOC_PATH = r".\documents\curated\general-principles-2021-01-21-final-CURATED.pdf"
# DOC_PATH = r".\documents\curated\Guidelines on Core Components of Infection Prevention and Control Programmes-CURATED.pdf"
# DOC_PATH = r".\documents\curated\Healthy-WHO-CURATED.pdf"
DOC_PATH = r".\documents\curated\Outpatient-Guide-508-CURATED.pdf" # single chapter

In [ ]:
# lets see the diffent police sizes in the document
def get_font_sizes(doc_path, page):
    doc = fitz.open(doc_path)
    page = doc.load_page(page)
    blocks = page.get_text("dict")["blocks"]
    font_sizes = set()
    for block in blocks:
        if block["type"] == 0:  # text block
            for line in block["lines"]:
                for span in line["spans"]:
                    font_sizes.add(span["size"])
    return sorted(font_sizes)

    

In [ ]:
font_sizes = get_font_sizes(DOC_PATH, 0)
print(font_sizes)
#print exepmle of the text with the different font sizes
def get_text_by_font_size(doc_path, page, font_size):
    doc = fitz.open(doc_path)
    page = doc.load_page(page)
    blocks = page.get_text("dict")["blocks"]
    texts = []
    for block in blocks:
        if block["type"] == 0:  # text block
            for line in block["lines"]:
                for span in line["spans"]:
                    if span["size"] == font_size:
                        texts.append(span["text"])
    return texts
for size in font_sizes:
    texts = get_text_by_font_size(DOC_PATH, 0, size)
    print(f"Font size: {size}, Example text: {texts[:3]}")

[11.0, 12.0, 18.0]
Font size: 11.0, Example text: ['Version 2.3 - September 2016', '6']
Font size: 12.0, Example text: ['Dedicate Resources to Infection Prevention ', '(Administrative Resources)', 'Infection prevention must be made a priority in ']
Font size: 18.0, Example text: ['FUNDAMENTAL ELEMENTS NEEDED TO PREVENT ', 'TRANSMISSION OF INFECTIOUS AGENTS IN ', 'OUTPATIENT SETTINGS']


In [ ]:
CHAPTER_FONT_SIZE = 18

In [ ]:
# lets save the chapeters font size for each document, in json
import json
def save_chapter_font_size(doc_path, font_size):
    # doc name the section before the first dot pdf
    doc_name = doc_path.split("\\")[-1].split(".")[0]
    data = {

        "doc_path": doc_name,
        "chapter_font_size": font_size
    }
    with open("chapter_font_sizes.json", "a") as f:
        json.dump(data, f)
        f.write("\n")
        

In [ ]:
# save_chapter_font_size(DOC_PATH, CHAPTER_FONT_SIZE)
# print(f"Chapter font size for {DOC_PATH} saved to chapter_font_sizes.json")

# END